# scrapy_当当网开启多条管道下载

接上一节

In [ ]:
# scrapy_dangdang_045/scrapy_dangdang_045/pipelines.py

# 使用pipeline管道下载数据
# 首先要在settings.py中开启管道
# ------------------------------
# settings.py:
# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
# ITEM_PIPELINES = {
#     # 管道可以有很多个  管道是有优先级的    优先级的范围是1-1000 值越小优先级越高
#    "scrapy_dangdang_045.pipelines.ScrapyDangdang045Pipeline": 300,
# }



# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter

# 如果想使用管道的话 那么就必须在settings中开启管道
class ScrapyDangdang045Pipeline:
    # 在爬虫文件开始之前就执行的方法
    def open_spider(self, spider):
        print('++++++++++++++++++++++++++')
        self.fp = open('book.json', 'w', encoding='utf-8')



    # item就是yield后面的book对象
    def process_item(self, item, spider):
        # 以下这种模式不推荐    因为每传递过来一个对象 就会打开一次文件 对文件的操作过于频繁
        # # (1) write 方法必须要写一个字符串，而不能是其他对象
        # # (2) w模式 会每个对象都打开一次文件 覆盖之前的内容
        # with open('book.json', 'a', encoding='utf-8') as fp:
        #     fp.write(str(item))

        # 推荐使用以下模式 在前面定义
        self.fp.write(str(item))

        return item
    

    # 在爬虫文件结束之后就执行的方法
    def close_spider(self, spider):
        self.fp.close()


import urllib.request
import os

# 多条管道开启
# (1) 定义管道类
# (2) 在settings.py中开启管道 
#    添加新的管道和优先级
#    "scrapy_dangdang_045.pipeline.DangDangDownloadPipeline": 301,

class DangDangDownloadPipeline:
    def process_item(self, item, spider):
        # 下载图片
        os.makedirs('books', exist_ok=True)
        url = 'http:' + item.get('src')
        filename = './books/' + item.get('name') + '.jpg'
        urllib.request.urlretrieve(url=url, filename=filename)
        return item